In [1]:
! py -m pip install --upgrade python-dotenv
! py -m pip install --upgrade llama-index
! py -m pip install --upgrade llama-index-llms-openai
! py -m pip install --upgrade llama-index-embeddings-openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached llama_index-0.12.8-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.4.1-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_cli-0.4.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.12.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.6.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached llama_index_llms_openai-0.3.12-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_multi_modal_llms_openai-0.4.1-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_program_openai-0.3.1-py3-none-any.whl.metadata (764 bytes)
  Using cached llama_index_question_gen_openai-0.3.0-py3-none-any.whl.metadata (783 bytes)
  Using cached llama_index_readers_file-0.4.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import nest_asyncio

openai_api_key = "sk-proj-g1ZDvCjAWM0A"
nest_asyncio.apply()

In [3]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [4]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
node_metadata = nodes[1].get_content(metadata_mode=True)
print(str(node_metadata))

page_label: 2
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-12-25
last_modified_date: 2024-12-25

Preprint
Figure 1: The software development SOPs between MetaGPT and real-world human teams.
In software engineering, SOPs promote collaboration among various roles. MetaGPT showcases
its ability to decompose complex tasks into specific actionable procedures assigned to various roles
(e.g., Product Manager, Architect, Engineer, etc.).
documents, design artifacts, flowcharts, and interface specifications. The use of intermediate struc-
tured outputs significantly increases the success rate of target code generation. Because it helps
maintain consistency in communication, minimizing ambiguities and errors during collaboration.
More graphically, in a company simulated by MetaGPT, all employees follow a strict and stream-
lined workflow, and all their handovers must comply with certain established standards. This reduces
the ri

#### Define LLM and Embedding Model

In [7]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embedding = OpenAIEmbedding(model="text-embedding-ada-002")

#### Sumary Index and Vector Index over the same data

`SummaryIndex` and `VectorStoreIndex` are specialized index types for handling different data storage and retrieval mechanisms, both designed to work with data that's been preprocessed into "nodes".  
  
`SummaryIndex` is designed to handle the summary or aggregation of documents. nodes are the collection of text data.  
  
`VectorStoreIndex` is designed to store and manage the vector embeddings that are used for tasks like semantic search or similarity search. 

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

#### Define Query Engines and Set Metadata

Both `summary_index` and `vector_index` are converted into `query engines` - which is responsible to processing queries and generating responses based on the data (nodes) that have been indexed. 

In [9]:
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async = True,
)
vector_query_engine = vector_index.as_query_engine()

In [12]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

#### Define Router Query Engine

In [16]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose= True
)

In [17]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through Standardized Operating Procedures (SOPs). It incorporates role specialization, workflow management, and efficient communication mechanisms to improve problem-solving capabilities. MetaGPT involves agents like Product Managers, Architects, Project Managers, Engineers, and QA Engineers, each with specific roles and responsibilities. Through a collaborative process, MetaGPT generates structured outputs like technical specifications, code files, and unit tests to streamline software development tasks. The document details the development process of a software application called the "Drawing App" using MetaGPT, covering requirements gathering, UI design, system architecture, implementation approach, testing, and task allocation.

In [18]:
print(len(response.source_nodes))

34


In [19]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper, which would likely include information on how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. By storing information in this global message pool, agents can retrieve required information without the need to inquire about other agents individually, thus enhancing communication efficiency.


In [22]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results show that MetaGPT effectively addresses challenges related to context utilization, code hallucinations, and information overload in software development. By accurately interpreting natural language descriptions, maintaining information validity, and focusing on granular tasks, MetaGPT mitigates issues such as incomplete code implementation, missing dependencies, and undiscovered bugs. Additionally, the use of a global message pool and a subscription mechanism helps manage information overload by streamlining communication and filtering out irrelevant contexts, thereby enhancing the relevance and utility of information in software development.
